In [3]:
# J'importe les bibliothèques nécessaires : requests pour réaliser des requêtes HTTP,
# BeautifulSoup pour analyser le HTML, pandas pour manipuler les données, et os pour gérer les opérations liées au système de fichiers.
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

# Je configure les en-têtes HTTP pour simuler une requête provenant d'un navigateur web moderne.
# Ceci est utile pour éviter d'être bloqué par les mécanismes de protection anti-scraping des sites web.
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Je spécifie le chemin du dossier local où je souhaite enregistrer les images téléchargées.
img_folder_path = 'D:/Images_champignons'
# Si le dossier spécifié n'existe pas, je le crée pour éviter des erreurs lors de la sauvegarde des fichiers.
if not os.path.exists(img_folder_path):
    os.makedirs(img_folder_path)

# J'initialise une session de requêtes HTTP, ce qui me permet de conserver des cookies entre les requêtes
# et d'améliorer la gestion des connexions HTTP.
session = requests.Session()
session.headers.update(headers)  # J'ajoute l'en-tête User-Agent à la session pour toutes les requêtes.

# Je définis l'URL de base du site web que je veux scraper.
base_url = 'https://mushroomobserver.org/'

# J'initialise une liste vide pour collecter les données que je vais extraire.
data = []

# Je démarre une boucle pour parcourir les pages du site. Ici, je limite l'exemple aux trois premières pages.
for i in range(5, 120):
    # Je construis l'URL de chaque page en ajoutant le numéro de page à l'URL de base.
    page_url = f'{base_url}{i}'
    # Je récupère un identifiant unique de la page à partir de son numéro.
    url_reference = str(i)
    # J'initialise une liste pour stocker les noms des images trouvées sur chaque page.
    img_names = []
    
    # J'essaie d'effectuer une requête GET pour récupérer le contenu de la page.
    try:
        response = session.get(page_url)
        response.raise_for_status()  # Je vérifie que la requête a réussi sans retourner d'erreur HTTP.
    except requests.exceptions.HTTPError as err:
        print(f"Erreur HTTP : {err}")  # En cas d'erreur, je l'affiche et passe à la page suivante.
        continue
    
    # J'utilise BeautifulSoup pour analyser le contenu HTML de la page.
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Je cherche dans le document HTML la balise <h1> avec l'ID 'title' pour extraire le nom du champignon.
    name_tag = soup.find('h1', id='title').find('i')
    # Si le nom est trouvé, je l'extrait, sinon j'utilise un nom par défaut.
    name = name_tag.get_text(strip=True) if name_tag else f'Unknown_{i}'
    
    # Je recherche toutes les balises <img> qui contiennent les images des champignons.
    img_tags = soup.find_all('img', class_='carousel-image')
    img_counter = 1
    for img_tag in img_tags:
        # Pour chaque image, j'extrait son URL.
        img_url = img_tag.get('data-src')
        if not img_url:
            continue  # Si aucune URL n'est trouvée, je passe à l'image suivante.
        
        try:
            # Je télécharge l'image en utilisant l'URL trouvée.
            img_data = session.get(img_url).content
            # Je génère un nom de fichier unique pour l'image.
            img_name = f"{url_reference}_{name.replace(' ', '_')}_{img_counter}.jpg"
            img_path = os.path.join(img_folder_path, img_name)  # Je construis le chemin complet du fichier image.
            
            # J'ouvre le fichier en mode écriture binaire et j'enregistre les données de l'image.
            with open(img_path, 'wb') as f:
                f.write(img_data)
            
            # J'ajoute le nom du fichier à la liste des noms d'images.
            img_names.append(img_name)
            img_counter += 1  # J'incrémente le compteur d'images.
        except Exception as e:
            print(f"Erreur lors de la sauvegarde de l'image : {e}")  # J'affiche les erreurs rencontrées lors de la sauvegarde des images.
    
    # Je crée un dictionnaire pour cette page contenant l'URL, le nom du champignon, et les noms des images.
    page_data = {'URL': page_url, 'Name': name}
    for j, img_name in enumerate(img_names, start=1):
        page_data[f'image_{j}'] = img_name
    
    # J'ajoute les données de la page à ma liste de données.
    data.append(page_data)

# Je convertis la liste de données en DataFrame pour une manipulation et une analyse plus facile.
df = pd.DataFrame(data)
# Je redéfinis les colonnes du DataFrame pour inclure toutes les images collectées.
max_images = max(len(d) for d in data) - 2
df = df.reindex(columns=['URL', 'Name'] + [f'image_{j}' for j in range(1, max_images + 1)])

# J'affiche le DataFrame pour vérifier les données extraites.
print(df)

# Je sauvegarde le DataFrame au format Excel pour une utilisation ultérieure.
output_path = 'D:/Images_champignons/mushroom_data.xlsx'
df.to_excel(output_path, index=False)

# Je confirme que les données ont été sauvegardées avec succès.
print(f'Données sauvegardées à {output_path}')


                                  URL                      Name  \
0      https://mushroomobserver.org/5              Xeromphalina   
1      https://mushroomobserver.org/6   Xeromphalina campanella   
2      https://mushroomobserver.org/7         Xerocomus zelleri   
3      https://mushroomobserver.org/8         Xerocomus zelleri   
4      https://mushroomobserver.org/9   Xerocomus subtomentosus   
..                                ...                       ...   
110  https://mushroomobserver.org/115                   Ramaria   
111  https://mushroomobserver.org/116                   Ramaria   
112  https://mushroomobserver.org/117  Ramaria rubribrunnescens   
113  https://mushroomobserver.org/118         Ramaria largentii   
114  https://mushroomobserver.org/119        Ramaria gelatinosa   

                                image_1                          image_2  \
0                  5_Xeromphalina_1.jpg                              NaN   
1       6_Xeromphalina_campanella_1.jpg    